In [ ]:
from spectrumx.client import Client

# Create an SDK client to download DigitalRF data from the SDS
sds = Client(
    host="sds.crc.nd.edu",
    env_config={"SDS_SECRET_TOKEN": "YOUR_SECRET_TOKEN"},
)
sds.dry_run = False
sds.authenticate()

# download the DigitalRF data from the SDS
results = sds.download(
    from_sds_path="/YOUR_PATH",
    to_local_path="data",
    overwrite=False,
)

In [ ]:
from digital_rf import DigitalRFReader
import h5py
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal.windows import gaussian
from scipy.signal import ShortTimeFFT
from drf_plot import specgram_plot # make sure drf_plot.py is in the same folder as this code

# set the data directory where the DigitalRF data is stored (top level)
data_dir = "data/files/YOUR_USERNAME/YOUR_PATH/"

reader = DigitalRFReader(data_dir)
channels = reader.get_channels()

for channel in channels:
    print(f"Processing channel: {channel}")
    start_sample, end_sample = reader.get_bounds(channel)

    with h5py.File(f"{data_dir}/{channel}/drf_properties.h5", "r") as f:
        sample_rate = f.attrs["sample_rate_numerator"] / f.attrs["sample_rate_denominator"]

    num_samples = int(sample_rate * 2)
    rf_data = reader.read_vector(start_sample, num_samples, channel)

    # Compute spectrogram
    window = gaussian(1000, std=100, sym=True)
    fft_size = 1024
    stfft = ShortTimeFFT(window, hop=500, fs=sample_rate, mfft=fft_size, fft_mode="centered")
    spectrogram = stfft.spectrogram(rf_data)

    # Convert to dB
    spectrogram_db = 10.0 * np.log10(spectrogram + 1e-12)

    # Auto compute zscale using logic from drf_plot.py
    Pss_ma = np.ma.masked_invalid(spectrogram_db)
    zscale_low = 30
    zscale_high = np.median(Pss_ma.max()) + 10.0
    zscale = (zscale_low, zscale_high)
    print(f"zscale low: {zscale_low}")
    print(f"zscale high: {zscale_high}")
    

    # Create extent for plotting
    extent = stfft.extent(num_samples)

    specgram_plot(
        data=spectrogram,
        extent=extent,
        log_scale=True,
        zscale=zscale,
        title=f"Spectrogram - {channel}"
    )